In [1]:
!pip install happytransformer
!pip install ipywidgets

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import pandas as pd

import time

from happytransformer import HappyTextClassification
happy_tc = HappyTextClassification(model_type="DISTILBERT", model_name="distilbert-base-uncased-finetuned-sst-2-english", num_labels=2)

07/14/2022 14:47:39 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [4]:
def get_prod_title(driver, prod, prod_name):
    try:
        names =prod.find_elements(By.XPATH,".//span[@class='a-size-medium a-color-base a-text-normal']")
        prod_name.append([name.text for name in names])
        
    except Exception as e:
        print("Prod name: ", e)
        prod_name.append([""])
        
    return prod_name

def get_prod_link_nb_reviews(driver, prod, prod_no_reviews, prod_link):
    try:
        if len(prod.find_elements(By.XPATH,".//span[@class='a-size-base s-underline-text']"))>0:
            nb_reviews = prod.find_elements(By.XPATH,".//span[@class='a-size-base s-underline-text']")
            reviews = prod.find_elements(By.XPATH,".//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style']")
            #print(len(reviews),len(nb_reviews))
                
            if len(reviews)>1:
                link=prod_link.append(reviews[0].get_attribute("href"))
            else:
                for rev in reviews:
                    link=rev.get_attribute("href")
                    prod_link.append(link)

            for review in nb_reviews:
                prod_no_reviews.append(review.text)
                             
        else:
            prod_no_reviews.append(["0"])
            prod_link.append([""])
                
    except Exception as e:
        print("Reviews error: ", e) 
    return prod_no_reviews, prod_link

def get_product_details(driver, pr_link, prod_price,category,manufacturer,description,shipping_link, delivery_span, sold_by, ships_from,all_reviews_link,all_reviews_body,all_prod_sent):    
    def get_price(driver, prod_price):
        try:
            if len(driver.find_elements(By.XPATH,".//span[@class='a-price a-text-price a-size-medium apexPriceToPay']"))>1:
                prices=driver.find_elements(By.XPATH,".//span[@class='a-price a-text-price a-size-medium apexPriceToPay']")
                #print(type(prices))
                prod_price.append([price.text for price in prices])
            else:
                #pricess=driver.find_elements(By.XPATH,".//div[@class='twisterTextDiv text']") # to get n-persons
                prices=driver.find_elements(By.XPATH,".//span[@class='a-size-mini olpWrapper']")
                prod_price.append([t.get_attribute("innerHTML") for t in prices])

        except Exception as e:
            print("Price error: ", e)
            prod_price.append([e])   
        return prod_price

    def get_category_chain(driver,category):
        try:
            #categories=driver.find_elements(By.XPATH,".//*[@id='wayfinding-breadcrumbs_container']")
            categories=driver.find_elements(By.XPATH,".//*[@id='wayfinding-breadcrumbs_feature_div']/ul/li")
            #print(len(categories))
            if len(categories)>1:
                category.append([cat.text for cat in categories if cat.text!='›'])
            else:
                category.append([""])

        except Exception as e:
            print("Category error: ", e)
            category.append([e]) 
        return category

    def get_manuf(driver,manufacturer):
        try:
            manu=driver.find_elements(By.XPATH,".//*[@id='bylineInfo_feature_div']")
            #print(len(manu))
            if len(manu)==1:
                manufacturer.append(" ".join(manu[0].text.split(" ")[2:]))
            else:
                manufacturer.append([""])

        except Exception as e:
            print("Manufacturer error: ", e)
            manufacturer.append([e])
        return manufacturer

    def get_description(driver, description):
        try: 
            text=driver.find_elements(By.XPATH,".//*[@id='feature-bullets']/ul/li/span")
            print("Description: ", len(text))
            if len(text)>0:
                description.append([sent.text for sent in text])
            else:
                description.append([""])

        except Exception as e:
            print("Description error: ", e)
            description.append([e])    
        return description

    def get_shipping(driver, shipping_link, delivery_span, sold_by, ships_from):
        try:
            shipping_options=driver.find_elements(By.XPATH,".//*[@id='buybox-see-all-buying-choices']/span/a")
            print("shipping info : ",len(shipping_options), type(shipping_options))

            out_of_stock=driver.find_elements(By.XPATH,".//*[@id='outOfStock']/div/div[1]/span[1]")
            print("out of stock: ", len(out_of_stock))

            select_size=driver.find_elements(By.XPATH,".//*[@id='partialStateBuybox']/div/div[1]/span[1]")
            print("select size first: ", len(select_size))

            if len(out_of_stock)>0:
                shipping_link.append(["No stock"])
                delivery_span.append(["No stock"])
                sold_by.append(["No stock"])
                ships_from.append(["No stock"])

            elif len(shipping_options)>0:    
                for i in shipping_options:
                    print("shipping link: ",i.get_attribute("href"))
                    shipping_info_link=i.get_attribute("href")
                    shipping_link.append(shipping_info_link)
                    time.sleep(5)
                    driver.get(shipping_info_link)
                    delivery_dates=driver.find_elements(By.XPATH,".//*[@id='mir-layout-DELIVERY_BLOCK-slot-PRIMARY_DELIVERY_MESSAGE_LARGE']/span/span")
                    print("delivery dates: ",len(delivery_dates), delivery_dates)

                    if len(delivery_dates)>0:
                        delivery_span.append([i.text for i in delivery_dates])
                        #print([i.text for i in delivery_dates])
                    else:
                        delivery_span.append([""])    

                    sold=driver.find_elements(By.XPATH,".//*[@id='aod-offer-shipsFrom']/div/div/div[2]/span")
                    #print("sold by: ", len(sold_by), sold_by)
                    if len(sold_by)>0:  
                        sold_by.append([i.text for i in sold])
                    else:
                        sold_by.append([""])


                    ships=driver.find_elements(By.XPATH,".//*[@id='aod-offer-shipsFrom']/div/div/div[2]/div/span")
                    #print("ships from: ",len(ships_from))

                    if len(ships_from)>0:
                        ships_from.append([i.text for i in ships])
                    else:
                        ships_from.append([""])

            # add elif when we have to select size to view price                        
            else:
                shipping_link.append([""])
                delivery_span.append([""])
                sold_by.append([""])
                ships_from.append([""])


        except Exception as e:
            print("Availability error: ", e)
            #availability.append([e])

        return shipping_link, delivery_span, sold_by, ships_from

    def get_reviews(driver, pr_link,all_reviews_link,all_reviews_body ):

        try:
            time.sleep(5)
            driver.get(pr_link)

            reviews_link=driver.find_elements(By.XPATH,".//*[@id='cr-pagination-footer-0']/a")
            print("reviews link ", len(reviews_link), reviews_link[0].get_attribute("href"))
            all_reviews_link.append([i.get_attribute("href") for i in reviews_link])

            time.sleep(5)
            driver.get(reviews_link[0].get_attribute("href"))

            reviews_body=driver.find_elements(By.XPATH,".//span[@data-hook='review-body']")        
            print("Reviews body: ", type(reviews_body),len(reviews_body))
            all_reviews_body.append([i.text for i in reviews_body])


        except Exception as e:
            print("reviews link: ",e)
            all_reviews_link.append([""])
            all_reviews_body.append([""])
            
        print("Finished extracting reviews ")
        return all_reviews_link,all_reviews_body
    
    def get_sentiment(all_reviews_body,all_prod_sent):
        print("Sentiment", len(all_reviews_body))
        prod_sent=[]
        for i in all_reviews_body:
            try:
                one_prod_sent = happy_tc.classify_text(i)
                #print(type(one_prod_sent),one_prod_sent.label, one_prod_sent.score)
                prod_sent.append([[one_prod_sent.label, one_prod_sent.score]])
            except Exception as e:
                #print(e)
                prod_sent.append(["The text is too big to be classified"])
                
        all_prod_sent.append(prod_sent)
        print("Sentiment completed")
        return all_prod_sent
            
    
    prod_price=get_price(driver, prod_price)
    category=get_category_chain(driver,category)
    manufacturer=get_manuf(driver,manufacturer)
    description=get_description(driver, description)
    shipping_link, delivery_span, sold_by, ships_from=get_shipping(driver, shipping_link, delivery_span, sold_by, ships_from)
    all_reviews_link,all_reviews_body=get_reviews(driver, pr_link,all_reviews_link,all_reviews_body )
    #print(all_reviews_body[-1])
    all_prod_sent=get_sentiment(all_reviews_body[-1],all_prod_sent)
    
    return prod_price,category,manufacturer,description,shipping_link, delivery_span, sold_by, ships_from,all_reviews_link,all_reviews_body,all_prod_sent


In [6]:

def scrape_amazon(search, search_pages):
    prod_name = []
    prod_link=[]

    prod_price = []
    prod_no_reviews = []
    prod_reviews=[]
    category=[]
    manufacturer=[]
    description=[]
    shipping=[]
    delivery_span=[]
    shipping_link=[]
    sold_by=[]
    ships_from=[]

    all_reviews_link=[]
    all_reviews_body=[]
    all_prod_sent=[]

    PATH = "C:\webdrivers\chromedriver.exe"
    driver = webdriver.Chrome(PATH)
    search_query = search.replace(' ', '+')
    
    for page in range(1,search_pages+1):
        original_link="https://www.amazon.com/s?k="+search_query+"&page="+ str(page)
        driver.get(original_link)
        #driver.maximize_window()
        prods = driver.find_elements(By.XPATH,'//div[@data-component-type="s-search-result"]')
        #print(len(prods))
        count=1
        for prod in prods:
            prod_name=get_prod_title(driver, prod, prod_name)
            prod_no_reviews, prod_link=get_prod_link_nb_reviews(driver, prod, prod_no_reviews, prod_link)

            print("Count: ", count)
            count+=1
            #if count==3:
                #break
            #break
        time.sleep(5)    
        #break

    for pr_link in prod_link:
        time.sleep(1)
        driver.get(pr_link)
        print(pr_link)  
        prod_price,category,manufacturer,description,shipping_link, delivery_span, sold_by, ships_from,all_reviews_link,all_reviews_body,all_prod_sent=get_product_details(driver, pr_link, prod_price,category,manufacturer,description,shipping_link, delivery_span, sold_by, ships_from,all_reviews_link,all_reviews_body,all_prod_sent)
    
    df_products=pd.DataFrame({'Name': prod_name, 'Link':prod_link, "Nb of Reviews":prod_no_reviews, "Price Range": prod_price, "Category_chain":category, "Manufacturer":manufacturer, "Description":description, "Shipping link": shipping_link,"Delivery Span":delivery_span, "Sold By":sold_by, "Ships From":ships_from, "All Reviews Link": all_reviews_link, "All Reviews Texts":all_reviews_body, "Reviews Sentiment": all_prod_sent})
    #print(df_products.head())
    df_products.to_csv("Result.csv", index=False)
    
scrape_amazon("camping tent", 2)

C:\Users\User\AppData\Local\Temp\ipykernel_5904\686714111.py:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


Count:  1
Count:  2
Count:  3
Count:  4
Count:  5
Count:  6
Count:  7
Count:  8
Count:  9
Count:  10
Count:  11
Count:  12
Count:  13
Count:  14
Count:  15
Count:  16
Count:  1
Count:  2
Count:  3
Count:  4
Count:  5
Count:  6
Count:  7
Count:  8
Count:  9
Count:  10
Count:  11
Count:  12
Count:  13
Count:  14
Count:  15
Count:  16
https://www.amazon.com/Coleman-4-Person-Camping-Sundome-Setup/dp/B019N9W7WC/ref=sr_1_1?keywords=camping+tent&qid=1657800910&sr=8-1#customerReviews
Description:  5
shipping info :  0 <class 'list'>
out of stock:  0
select size first:  1
reviews link  1 https://www.amazon.com/Coleman-Sundome-Camping-Tent/product-reviews/B09WDKXRHP/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
Reviews body:  <class 'list'> 10
Finished extracting reviews 
Sentiment 10
Sentiment completed
https://www.amazon.com/Coleman-2000018016-4-Person-Instant-Cabin/dp/B004E4AVY8/ref=sr_1_2?keywords=camping+tent&qid=1657800910&sr=8-2#customerReviews
Description:  6
shipping info

Reviews body:  <class 'list'> 10
Finished extracting reviews 
Sentiment 10
Sentiment completed
https://www.amazon.com/ALPS-Mountaineering-Lynx-4-Person-Tent/dp/B0842S9YHP/ref=sr_1_16?keywords=camping+tent&qid=1657800910&sr=8-16#customerReviews
Description:  8
shipping info :  0 <class 'list'>
out of stock:  1
select size first:  0
reviews link  1 https://www.amazon.com/ALPS-Mountaineering-Lynx-4-Person-Tent/product-reviews/B0842S9YHP/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
Reviews body:  <class 'list'> 10
Finished extracting reviews 
Sentiment 10
Sentiment completed
https://www.amazon.com/Coleman-Camping-Skylodge-Person-Evergreen/dp/B08KGSV9QT/ref=sr_1_17?keywords=camping+tent&qid=1657800925&sr=8-17#customerReviews
Description:  8
shipping info :  0 <class 'list'>
out of stock:  1
select size first:  0
reviews link  1 https://www.amazon.com/Coleman-Camping-Skylodge-Person-Evergreen/product-reviews/B08KGSV9QT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_revi